In [1]:
'''This Program reads video files and classify them into following activties
Hand washing, Medical Sink interaction, items placed, others: water pouring / running'''
#%matplotlib inline
import cv2
import numpy as np
import pandas as pd
import os
from IPython.display import Image
from IPython.display import display
from matplotlib import pyplot as plt

#Video reading my cv2 stopped working since ffmpeg has some issue on my system. So using skvideo to read.
from skvideo.io import VideoCapture

In [2]:
#Give the path to folder with all the videos
path = '/Users/homw/Documents/MSDS16/IndStudy/videos/'

In [3]:
#Creates a csv file with all the details of videos by parsing the name
roomNo, dayOfWeek, month, day, hours, minutes, seconds, year = [],[],[],[],[],[],[],[]
filename = []
for f in os.listdir(path):
    if f.endswith("mp4"):
        roomNo.append(f[:4])
        dayOfWeek.append(f[7:10])
        month.append(f[11:14])
        day.append(f[15:17])
        hours.append(f[18:20])
        minutes.append(f[21:23])
        seconds.append(f[24:26])
        year.append(f[27:31])
        filename.append(f[:-4])

df = pd.DataFrame({"File_Name": filename,"RoomNo": roomNo, "Day_of_week": dayOfWeek, "Month": month, "Day": day, "Time_hh":hours, 
                   "Time_mm": minutes,"Time_ss": seconds, "Year": year})
df.to_csv("video_data.csv", index = False)

In [4]:
#vid=cv2.VideoCapture(os.path.join(path,data["File_Name"][0]+".mp4"))
def get_first_last_diff(file_path):
    vid = VideoCapture(file_path)
    success, frame_0 = vid.read()
    while success:
        success, frame = vid.read()
        if success:
            frame_n = frame

    frame_0 = cv2.cvtColor(frame_0, cv2.COLOR_RGB2GRAY)
    frame_n = cv2.cvtColor(frame_n, cv2.COLOR_RGB2GRAY)
    diff = cv2.absdiff(frame_n, frame_0)
    _,diff = cv2.threshold(diff,80,255,cv2.THRESH_BINARY)

    #erode
    kernel = np.ones((5,5), np.uint8)
    diff = cv2.erode(diff, kernel, iterations=1)
    return np.count_nonzero(diff)

In [5]:
def glove_detector(fpath):
    vid = VideoCapture(os.path.join(fpath))
    
    success, frame_0 = vid.read()
    frame_0g = cv2.cvtColor(frame_0, cv2.COLOR_RGB2GRAY)
    
    kernel = np.ones((5,5), np.uint8)
    count_purple = 0
    purple = []
    while success:
        success, frame = vid.read()
        if success:
            
            frame_ng = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            diff = cv2.absdiff(frame_ng, frame_0g)
            _,diff = cv2.threshold(diff,80,255,cv2.THRESH_BINARY)
            #erode
            diff = cv2.erode(diff, kernel, iterations=1)
            #print("the difference is {}".format(np.count_nonzero(diff)))
            if np.count_nonzero(diff) > 2000:
                hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
                # define range of purple color in HSV - color of gloves
                lower_purple = np.array([120,180,120])
                upper_purple = np.array([140,200,140])

                # Threshold the HSV image to get only purple colors
                mask = cv2.inRange(hsv, lower_purple, upper_purple)

                # Bitwise-AND mask and original image
                res = cv2.bitwise_and(frame,frame, mask= mask)
                count_purple = np.count_nonzero(res)
            if count_purple > 500:
                purple.append(1)
                #Reset
                count_purple = 0 
            else:
                purple.append(0)
    
    return purple

In [8]:
def handwash_detector(fpath):
    
    #fpath = os.path.join(path, "3132 - Fri Jan 15 15-47-38 2016.mp4")
    vid = VideoCapture(fpath)
    handwash = []
    count_fg_f = []
    count_fg_l = []
    #Get first and last frames of the video
    success, frame_first = vid.read()
    frame_first = cv2.cvtColor(frame_first, cv2.COLOR_BGR2GRAY)
    while success:
        success, frame = vid.read()
        if success:
            frame_last = frame

    frame_last = cv2.cvtColor(frame_last, cv2.COLOR_BGR2GRAY)
    diff_l_f = cv2.absdiff(frame_last, frame_first)
    _,diff_l_f = cv2.threshold(diff_l_f,80,255,cv2.THRESH_BINARY)

    #reset the indexer to the first frame
    vid = VideoCapture(fpath)
    success, frame = vid.read()
    while success:
        success, frame = vid.read()
        if success:
            frame_ng = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            diff_n_f = cv2.absdiff(frame_ng, frame_first)
            diff_n_l = cv2.absdiff(frame_ng, frame_last)
            _,diff_n_f = cv2.threshold(diff_n_f,80,255,cv2.THRESH_BINARY)
            _,diff_n_l = cv2.threshold(diff_n_l,80,255,cv2.THRESH_BINARY)
            #erode
            kernel = np.ones((5,5), np.uint8)
            diff_n_f = cv2.erode(diff_n_f, kernel, iterations=1)
            diff_n_l = cv2.erode(diff_n_l, kernel, iterations=1)
            count_fg_f.append(np.count_nonzero(diff_n_f))
            count_fg_l.append(np.count_nonzero(diff_n_l))
            #print("first %d last %d"%(np.count_nonzero(diff_n_f), np.count_nonzero(diff_n_l)))
    count_fg = [1 if p>5000 and q>5000 else 0 for p,q in zip(count_fg_f,count_fg_l)]
    return(count_fg)

In [6]:
#Reads files from video_data.csv and Classifies them into following classes
# Hand washing, Medical Sink interaction, items placed, others: water pouring / running
#Writes a csv with classfied activities
def classify_videos():
    data = pd.read_csv("video_data.csv", delimiter=",")
    Act_ItemsPlaced = list()
    Act_MedSink = list()
    Act_HandWash = list()
    for idx, row in data.iterrows():
        f = row["File_Name"]
        fpath = f+".mp4"
        fpath = os.path.join(path,fpath)

        #Detect Activity: Items placed / removed
        diff = get_first_last_diff(fpath)
        #diff=0
        #print("diff for %s is %d" %(f,diff))

        # classify activity as items placed if the difference is more than 100
        if diff > 100:
            Act_ItemsPlaced.append(1)
        else:
            Act_ItemsPlaced.append(0)

        #Detect Activity: Medical sink interaction
        # if purple gloves detected, classfy as med-sink interaction
        count_glove_frames = np.sum(glove_detector(fpath))
        #print("glove frames count for %s is %d" %(f,count_glove_frames))
        count_handwash_frames = 0
        if count_glove_frames > 20:
            Act_MedSink.append(1)
        else:
            Act_MedSink.append(0)

            #Detect Activity: Hand washing
            #the videos that are not in MedSink category are potential candidates for handwash
            count_handwash_frames = np.sum(handwash_detector(fpath))

        #print("handwash frames count for %s is %d" %(f,count_handwash_frames))

        if count_handwash_frames > 50:
            Act_HandWash.append(1)
        else:
            Act_HandWash.append(0)
    Act_Others = [0 if p+q+r > 0 else 1 for p,q,r in zip(Act_MedSink,Act_ItemsPlaced,Act_HandWash)]
    data["Act_MedSink"] = Act_MedSink
    data["Act_ItemsPlaced"] = Act_ItemsPlaced
    data["Act_HandWash"] = Act_HandWash
    data["Act_Others"] = Act_Others
    data.to_csv("video_classification.csv", ",")
    print("%d videos classified successfully" %(len(data)))

In [9]:
#call the classification function
classify_videos()

21 videos classified successfully
